## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc #garbage collection 감시
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
import seaborn as sns
from functools import partial
from bayes_opt import BayesianOptimization
from lightgbm import LGBMClassifier

warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    min  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)

# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [2]:
train = pd.read_csv("data/quality 수정.csv")
test = pd.read_csv("data/quality 수정_test.csv")

In [3]:
train.columns

Index(['Unnamed: 0', 'user_id', 'model_nm', 'update', 'err_1', 'err_2',
       'err_3', 'err_4', 'err_5', 'err_6', 'err_7', 'err_8', 'err_9', 'err_10',
       'err_11', 'err_12', 'err_13', 'err_14', 'err_15', 'err_16', 'err_17',
       'err_18', 'err_19', 'err_20', 'err_21', 'err_22', 'err_23', 'err_24',
       'err_25', 'err_26', 'err_27', 'err_28', 'err_30', 'err_31', 'err_32',
       'err_33', 'err_34', 'err_35', 'err_36', 'err_37', 'err_38', 'err_39',
       'err_40', 'err_41', 'err_42', 'fw_rank', 'q_meanmax_0', 'q_meanmax_1',
       'q_meanmax_2', 'q_meanmax_5', 'q_meanmax_6', 'q_meanmax_7',
       'q_meanmax_8', 'q_meanmax_9', 'q_meanmax_10', 'q_meanmax_11',
       'q_meanmax_12', 'q_chg_0', 'q_chg_1', 'q_chg_2', 'q_chg_5', 'q_chg_6',
       'q_chg_11', 'label'],
      dtype='object')

In [5]:
train_x = train.drop(columns=["label","Unnamed: 0","user_id"])

train_y = train["label"]

test_x = train.drop(columns=["Unnamed: 0","user_id"])

In [11]:
def lgbm_cv(colsample_bytree, 
            learning_rate,
            max_depth,
            n_estimators,
            num_leaves,
          x_data=None, y_data=None, n_splits=5, output='score'):
    
    score = 0
    kf = KFold(n_splits=n_splits, random_state=342)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index,:], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index,:], y_data[valid_index]

        model = LGBMClassifier(
            colsample_bytree = colsample_bytree,
            learning_rate = learning_rate,
            max_depth = int(max_depth),
            n_estimators = int(n_estimators),
            num_leaves = int(num_leaves),
            objective = "binary"
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        prob = model.predict(x_valid)
        pred = np.where(prob > 0.5, 1, 0)
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [16]:
func_fixed = partial(lgbm_cv, x_data=train_x, y_data=train_y, n_splits=5, output="score")
regBO = BayesianOptimization(
    func_fixed, 
    {
         'colsample_bytree' : (0.1,1),
         'learning_rate' : (0.01, 0.1),
         "max_depth" : (0,1000),
         "n_estimators" : (100,200),
         "num_leaves" : (0,600),
    }, random_state=231)

regBO.maximize(init_points=5, n_iter=100)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | num_le... |
-------------------------------------------------------------------------------------
|  1        |  0.7217   |  0.8033   |  0.06258  |  426.7    |  131.5    |  496.1    |
|  2        |  0.713    |  0.9131   |  0.0135   |  915.2    |  106.1    |  108.4    |
|  3        |  0.7167   |  0.1651   |  0.05129  |  708.4    |  148.3    |  15.27    |
|  4        |  0.7152   |  0.1602   |  0.04833  |  219.2    |  181.0    |  84.03    |
|  5        |  0.7214   |  0.7761   |  0.03512  |  949.1    |  188.5    |  366.1    |
|  6        |  0.718    |  0.8863   |  0.02317  |  432.5    |  134.4    |  504.1    |
|  7        |  0.7204   |  0.9564   |  0.03725  |  424.9    |  124.2    |  496.8    |
|  8        |  0.7214   |  0.9028   |  0.05555  |  949.9    |  188.5    |  367.3    |
|  9        |  0.7206   |  0.9254   |  0.08167  |  418.3    |  135.8    |  488.5    |
|  10       |  0.7196   |  0.6481   |  0.08483  |  946

KeyboardInterrupt: 

In [ ]:
params = regBO.max['params']

In [ ]:
params

{'colsample_bytree': 0.9532847935680113,
 'learning_rate': 0.09278453605939078,
 'max_depth': 578.1033906895178,
 'n_estimators': 39.602849442683194,
 'num_leaves': 413.07521762936597}

In [ ]:
models = lgbm_cv(
    params['colsample_bytree'], 
    params['learning_rate'],
    params["max_depth"],
    params["n_estimators"],
    params["num_leaves"],
    x_data=train_x, y_data=train_y, n_splits=5, output='model')

# 5. 제출 파일 생성

In [ ]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [ ]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [ ]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532648/16532648 [00:40<00:00, 404165.80it/s]

(14999, 42)


In [ ]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_x)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [ ]:
pred_ensemble

array([[1. ],
       [0. ],
       [0. ],
       ...,
       [0.6],
       [1. ],
       [0.2]])

In [ ]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [ ]:
SUB_PATH = '/content/drive/MyDrive/YBIGTA/21-1 겨울방학/lg 품질변화 공모전/quality_change_prediction/submissions/'
sample_submssion.to_csv(SUB_PATH+"dacon_baseline_0111(3).csv", index = False)
sample_submssion

,user_id,problem
0,30000,1.0
1,30001,0.0
2,30002,0.0
3,30003,1.0
4,30004,1.0
...,...,...
14994,44994,0.0
14995,44995,0.0
14996,44996,0.6
14997,44997,1.0
